# Practical 7: Finite difference methods for ordinary differential

equations

Computational Finance with Python

[Alet Roux](https://www.york.ac.uk/maths/staff/alet-roux/) ([Department
of Mathematics](https://maths.york.ac.uk), University of York)

Click on the following to open this file in Google Colab:

<figure>
<a
href="https://colab.research.google.com/github/aletroux/comp-finance-python/blob/main/practicals/07_finite_difference_ODE_prac.ipynb"><img
src="https://colab.research.google.com/assets/colab-badge.svg"
alt="Open In Colab" /></a>
<figcaption>Open In Colab</figcaption>
</figure>

The aim of this practical is to explore finite difference methods for
approximating the solutions to initial value problems for ordinary
differential equations. The example that we will use is the Ricatti
equations that appear in formulae for bond pricing in affine term
structure models such as the Merton, Vasicek (1977) and Cox, Ingersoll,
and Ross (1985) models. These models are covered in a different module
(*Interest Rate Modelling*), and more information can be found in the
books of McInerney and Zastawniak (2015) (Chapter 3) and Brigo and
Mercurio (2007) (Section 3.2.4).

After conversion from a final value problem to an initial value problem,
the Ricatti differential equation takes the form
$$ \frac{\partial y}{\partial t}(t) = -\tfrac{1}{2}\gamma(t)y^2(t) + \lambda(t) y(t) + 1 $$
for all $t\in[0,T]$, with initial value $y(0) = 0$. The value of the
functions $\gamma$ and $\lambda$ depend on the model.

# 1. Vasicek model

We first take $\gamma(t) = 0$ and $\lambda(t)=-0.5$ for all $t$, so the
initial value problem becomes
$$ \begin{aligned}\frac{\partial y}{\partial t}(t) &= -0.5 y(t) + 1, & y(0) &= 0.\end{aligned} $$

The forward Euler method for approximating $y(1)$ with $N$ steps works
as follows:

1.  Set $T = 1$, $\Delta t = T/N$ and $t_k = k\Delta t$ for
    $k=0,\ldots,N$.
2.  Set $y_0 = 0$.
3.  For $k = 0, \ldots, N-1$, determine $y_{k+1}$ from
    $$\tfrac{1}{\Delta t}(y_{k+1} - y_k) = -0.5 y_k + 1,$$ in other
    words, $$y_{k+1} = y_k( 1 - 0.5\Delta t) + \Delta t.$$

The backward Euler method works the same, except in step 3 the
difference equation is replaced by
$$\tfrac{1}{\Delta t}(y_{k+1} - y_k) = -0.5 y_{k+1} + 1,$$ and the
iterative step becomes
$$y_{k+1} = \frac{y_k + \Delta t}{1 + 0.5\Delta t}.$$

The following code approximates $y(1)$ with $N=10$ steps and compares
the results of the forward and backward Euler methods graphically with
each other, and with the theoretical solution
$$ y(t) = 2\left(1- e^{-0.5 t}\right). $$ Study the code and the results
carefully. Vary the values of `N` and `T` to study the quality of the
approximations empirically.

In [1]:
import numpy as np

T = 1
N = 10
dt = T/N

# time steps for finite difference approximation
t = np.linspace(0,T,N+1)

# finite difference approximation - forward Euler
y_forward = np.zeros(N+1)
mult_forward = 1-0.5*dt
for k in range(N):
    y_forward[k+1] = y_forward[k]*mult_forward + dt

# finite difference approximation - backward Euler
y_backward = np.zeros(N+1)
div_backward = 1 + 0.5*dt
for k in range(N):
    y_backward[k+1] = (y_backward[k] + dt) / div_backward

print("Forward Euler approximation:",y_forward[-1])
print("Backward Euler approximation:",y_backward[-1])

# now plot approximation and theoretical solution
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize = (12,7))

# theoretical solution
theory_t = np.linspace(0,T,N**2)
theory_y = 2*(1-np.exp(-0.5*theory_t))
ax.plot(theory_t, theory_y, label = "Theoretical solution")
print("Theoretical value:",theory_y[-1])

# approximation
ax.plot(t, y_forward, marker = '.', label = "Forward Euler approximation")
ax.plot(t, y_backward, marker = 'x', label = "Backward Euler approximation")

# plot cosmetics
ax.set(title = "Finite difference approximation of Riccati equation for Vasicek model")
ax.set(xlabel='$t$', xlim=(0, T))
ax.xaxis.grid(True)
ax.yaxis.grid(True)
ax.legend(); # the semicolon prevents unwanted text from being displayed

<span class="theorem-title">**Exercise 1**</span> Implement the
Crank-Nicolson finite difference scheme for this initial value problem.
Use the same data ($T=1$ and $N=10$) as above.

Compare your results with the numerical results from the forward and
backward Euler methods. Which method would you recommend in this case?

# 2. Cox-Ingersoll-Ross model

Let us now take $\gamma = 0.4$ and $\lambda=-0.5$, so the initial value
problem becomes
$$ \begin{aligned}\frac{\partial y}{\partial t}(t) &= -0.2 y^2(t) - 0.5 y(t) + 1, & y(0) &= 0.\end{aligned} $$
A unique solution exists and it is known that $y(t)\ge0$ for all $t$
(from qualitative properties of the model).

## 2.1 Forward Euler method

<span class="theorem-title">**Exercise 2**</span> Use the forward Euler
method to approximate $y(1)$.

Your code should produce a numerical result, rounded to 4 decimal
places.

Test your code with $N=10$, $N=100$, $N=1000$ and $N=10000$.

## 2.2 Backward Euler method

The backward Euler method for approximating $y(1)$ with $N$ steps works
as follows:

1.  Set $T = 1$, $\Delta t = T/N$ and $t_k = k\Delta t$ for
    $k=0,\ldots,N$.
2.  Set $y_0 = 0$.
3.  For $k = 0, \ldots, N-1$, determine $y_{k+1}$ from
    $$\tfrac{1}{\Delta t}(y_{k+1} - y_k) = -0.2 y_{k+1}^2 - 0.5 y_{k+1} + 1.$$

After rearrangement, the equation in step 3 becomes
$$0.2 \Delta t y_{k+1}^2 + (1 + 0.5 \Delta t)y_{k+1} - y_k - \Delta t = 0.$$
It is not possible to make $y_{k+1}$ the subject of this equation, but
we can observe that this is a quadratic in $y_{k+1}$, and therefore the
formula for the roots of a quadratic can be used to solve for $y_{k+1}$.
These roots are
$$ \frac{1}{0.4\Delta t}\left(-(1 + 0.5 \Delta t)\pm \sqrt{d}\right),$$
where the discriminant $d$ is
$$ d = (1 + 0.5 \Delta t)^2 + 0.8 \Delta t (y_k + \Delta t). $$ Notice
that if $y_k\ge 0$, then two facts hold true about $d$:

-   $d>0$ because $\Delta t>0$. This means that the quadratic has two
    real-valued roots.
-   $d^2 \ge 1 + 0.5 \Delta t$. This means that at least one of the
    roots of the quadratic is nonnegative. Taking this root to be
    $y_{k+1}$, we obtain from $y_k\ge 0$ an approximation
    $y_{k+1}\ge 0$.

Combining this with the initial value $y_0=0$, we can construct an
approximation $y_0,\ldots,y_N$ such that $y_k\ge0$ for all $k$, which
matches the property $y(t)\ge0$ of the theoretical solution.

The following code implements the backward Euler method by calculating
the root directly, as above.

In [4]:
import math

# finite difference approximation - backward Euler
def backward_Euler (N):
    """Performs backward Euler approximation for CIR Riccati equation"""
    dt = T/N
    y = np.zeros(N+1)

    #pre-calculate some values that are used often
    coeff1 = 1 + 0.5*dt
    coeff1_sq = coeff1**2
    coeff2 = 0.8*dt
    coeff3 = 0.4*dt

    for k in range(N):
        d = coeff1_sq + coeff2*(y[k] + dt)
        y[k+1] = (math.sqrt(d) - coeff1)/coeff3
        
    return y

for N in [10, 100, 1000, 10000]:
    backward = backward_Euler(N)
    print(f"Backward Euler approximation of y(1) with {N} steps:", round(backward[-1], 4))

It is also possible to calculate the root using the `numpy.polynomial`
module (NumPy Developers (2022)), in particular the
`numpy.polynomial.polynomial` convenience class for power series. The
following code cell provides some basic usage.

In [5]:
import numpy.polynomial as nppoly

a = 2
b = 3
c = 1

# create the polynomial a + bx + cx^2
poly = nppoly.Polynomial([a, b, c])

# display it
print("Polynomial:", poly)

# access its degree - should be 2 as it's a quadratic
print("Degree:", poly.degree())

# calculate its roots
print("Roots:", poly.roots())

The following code uses this module for the finite difference
approximation.

In [6]:
# finite difference approximation - backward Euler
def backward_Euler (N):
    """Performs backward Euler approximation for CIR Riccati equation"""
    dt = T/N
    y = np.zeros(N+1)

    #pre-calculate some values that are used often
    coeff1 = 1 + 0.5*dt
    coeff2 = 0.2*dt

    for k in range(N):
        # create quadratic
        quadratic = nppoly.Polynomial([-y[k] - dt, coeff1, coeff2])

        # take biggest root
        y[k+1] = max(quadratic.roots())
   
    return y

for N in [10, 100, 1000, 10000]:
    backward = backward_Euler(N)
    print(f"Backward Euler approximation of y(1) with {N} steps:", round(backward[-1], 4))

## 2.3 Crank-Nicolson method

<span class="theorem-title">**Exercise 3**</span> Use the Crank-Nicolson
method to approximate $y(1)$ in the Riccati initial value problem
associated with the Cox-Ingersoll-Ross model.

Use your experience with the forward and backward Euler methods to
choose a suitable value for $N$ that approximates $y(1)$ accurately to
four decimal places.

# References

Brigo, Damiano, and Fabio Mercurio. 2007. *Interest Rate Models—Theory
and Practice: With Smile, Inflation and Credit*. Second edition.
Springer Finance. Springer.
<https://yorsearch.york.ac.uk/permalink/f/1d5jm03/44YORK_ALMA_DS51246890530001381>.

Cox, John C., Jonathan E. Ingersoll, and Stephen A. Ross. 1985. “A
Theory of the Term Structure of Interest Rates.” *Econometrica* 53 (2).
<https://yorsearch.york.ac.uk/permalink/f/7htm32/TN_cdi_crossref_primary_10_2307_1911242>.

McInerney, Daragh, and Tomasz Zastawniak. 2015. *Stochastic Interest
Rates*. Cambridge University Press.
<https://yorsearch.york.ac.uk/permalink/f/1d5jm03/44YORK_ALMA_DS21267964750001381>.

NumPy Developers. 2022. “Polynomials.”
<https://numpy.org/doc/stable/reference/routines.polynomials.html#>.

Vasicek, Oldrich. 1977. “An Equilibrium Characterization of the Term
Structure.” *Journal of Financial Economics* 5 (2): 177–88.
<https://doi.org/10.1016/0304-405X(77)90016-2>.